In [1]:
import torch
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from sklearn.model_selection import train_test_split

In [2]:
def showImage(img):
    show_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(show_img)
    plt.axis('off')
    plt.show()

In [3]:
if torch.cuda.is_available():
    device=torch.device(type='cuda',index=0)
else:
    device=torch.device(type='cpu',index=0)
print(device)

cpu:0


In [4]:
batch_size = 32
learning_rate = 0.001
epochs = 10

In [5]:
import os
import cv2
import numpy as np
import torch


base_dir = 'R:\\NIRMA SEM-V\\cvdl\\Digit_Recognition_using_CNN\\UCMerced_LandUse\\Images'

list_folders = os.listdir(base_dir)

images = []
labels = []

label_map = {folder: idx for idx, folder in enumerate(list_folders)}

print(label_map)

for folder in list_folders:
    folder_path = os.path.join(base_dir, folder)
    list_images = os.listdir(folder_path)
    
    for image_name in list_images:
        image_path = os.path.join(folder_path, image_name)
        img = cv2.imread(image_path)
        
        if img is None:
            print(f'Image {image_name} not loaded in folder {folder}')
            continue
        
        img = cv2.resize(img, (256, 256))
        img = img / 255.0  
        
        img_tensor = torch.from_numpy(np.transpose(img, (2, 0, 1))).float()   
        images.append(img_tensor)
        labels.append(label_map[folder])

# Convert the lists to Tensors
images = torch.stack(images)
labels = torch.tensor(labels, dtype=torch.int64)

print(images.shape, labels.shape)
print(images[0].type(), labels[0].type())


{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
torch.Size([2100, 3, 256, 256]) torch.Size([2100])
torch.FloatTensor torch.LongTensor


In [6]:
#spliting into train and test
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

train_dataset = torch.utils.data.TensorDataset(train_images, train_labels,)
test_dataset = torch.utils.data.TensorDataset(test_images, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)
print(len(train_loader), len(test_loader))

torch.Size([1680, 3, 256, 256]) torch.Size([420, 3, 256, 256]) torch.Size([1680]) torch.Size([420])
53 14


In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1=nn.Conv2d(3,32,kernel_size=3,stride=1,padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu=nn.ReLU()

        self.conv2=nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
        

        self.conv3=nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1)
        
        
        self.dropout = nn.Dropout(0.2)
   
        self.feature_size = self._get_conv_output((3, 256, 256))

        self.fc1 = nn.Linear(self.feature_size, 128)
        self.fc2 = nn.Linear(128, 21)

    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    def _forward_features(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        return x

    def forward(self, x):
        x=self.conv1(x)
        x=self.relu(x)
        x=self.pool(x)

        x=self.conv2(x)
        x=self.relu(x)
        x=self.pool(x)

        x=self.conv3(x)
        x=self.relu(x)
        x=self.pool(x)

        # x = self._forward_features(x)
        x = x.view(x.size(0), -1)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.dropout(x)
        # x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x   

In [20]:
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    acc = 0  # Keep track of cumulative accuracy
    total_loss = 0  # Keep track of total loss
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y.long())
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update accuracy and loss
        acc += (pred.argmax(1) == y).type(torch.float).sum().item()
        total_loss += loss.item()
        
        if batch % 100 == 0:
            loss_val = loss.item()
            current = batch * len(X)
            print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")
    
    # Calculate and print average accuracy and loss for the epoch
    acc /= size
    avg_loss = total_loss / len(dataloader)
    print(f"Accuracy for that epoch: {(100*acc):>7f}, Avg loss: {avg_loss:>7f}")
    
    return acc

In [22]:
def test_one_epoch(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
model = CNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
epochs=5
for i in range(epochs):
    print("Epoch No:", i + 1)
    train_one_epoch(train_loader, model, loss_fn, optimizer)

test_one_epoch(test_loader, model, loss_fn)

Epoch No: 1
loss: 2.725936  [    0/ 1680]
Accuracy: 0.223214, Avg loss: 2.522833
Epoch No: 2
loss: 1.857868  [    0/ 1680]
Accuracy: 0.397024, Avg loss: 2.002101
Epoch No: 3
loss: 1.778817  [    0/ 1680]
Accuracy: 0.572619, Avg loss: 1.411816
Epoch No: 4
loss: 0.904564  [    0/ 1680]
Accuracy: 0.692857, Avg loss: 1.021756
Epoch No: 5
loss: 0.597796  [    0/ 1680]
Accuracy: 0.804167, Avg loss: 0.629551
Test Accuracy: 50.2%, Avg loss: 0.066042 

